# Block Spin RG

We implement block-spin RG for the 1d Ising Model, and calculate $J$ and $h$ for several RG steps.

In [ ]:
using LinearAlgebra
using Random
using Statistics
using Optim
using ForwardDiff

#####################
# 1. Model & Indices
#####################

struct IsingModel1D
    lattice_size::Int
    J::Float64      # nearest-neighbor coupling
    h::Float64      # magnetic field
end

function make_indices(N::Int)
    ip1 = [2:N; 1]         # i+1 with periodic BC
    im1 = [N; 1:N-1]       # i-1 with periodic BC
    return ip1, im1
end

#####################
# 2. Fast Pseudo-Likelihood (nearest-neighbor only)
#####################

function neg_log_pl_fast(θ, data, ip1, im1)
    J, h = θ
    N = length(ip1)
    Tt = typeof(J)

    nbrs  = Vector{Int8}(undef, N)
    field = Vector{Tt}(undef, N)
    denom = Vector{Tt}(undef, N)
    loss  = zero(Tt)

    @inbounds for s in data
        for i in 1:N
            nbrs[i]  = s[ip1[i]] + s[im1[i]]
            field[i] = J * nbrs[i] + h
            denom[i] = one(Tt) + exp(-2 * s[i] * field[i])
        end
        loss += sum(log.(denom))
    end

    return loss
end

function estimate_pl(data::Vector{Vector{Int8}}; θ0 = [0.0, 0.0])
    # infer lattice size
    N = length(data[1])

    # build periodic-BC neighbor indices
    ip1, im1 = make_indices(N)

    # objective closure
    obj = θ -> neg_log_pl_fast(θ, data, ip1, im1)

    # optimize
    res = optimize(obj, θ0, BFGS(); autodiff = :forward)
    return Optim.minimizer(res)  # (J_hat, h_hat)
end

#####################
# 3. MCMC Sampler (nearest-neighbor only)
#####################

function total_energy(spins::Vector{Int8}, m::IsingModel1D)
    N = m.lattice_size
    E = 0.0
    # nearest-neighbor interactions (open chain here; change to periodic if you like)
    for i in 1:N-1
        E -= m.J * spins[i] * spins[i+1]
    end
    # magnetic field term
    for i in 1:N
        E -= m.h * spins[i]
    end
    return E
end

function mcmc_samples(model::IsingModel1D, T::Float64;
                      total_steps::Int=100_000,
                      burn_in::Int=20_000,
                      thin::Int=5,
                      batch_size::Int=4)
    N = model.lattice_size
    spins = Int8.(rand([-1, 1], N))
    E     = total_energy(spins, model)

    # burn-in
    for _ in 1:burn_in
        idxs = randperm(N)[1:batch_size]
        old  = copy(spins)
        for i in idxs
            spins[i] = -spins[i]
        end
        dE = total_energy(spins, model) - E
        if dE <= 0 || rand() < exp(-dE/T)
            E += dE
        else
            spins .= old
        end
    end

    # collect thinned samples
    n_rec   = div(total_steps - burn_in, thin)
    samples = Vector{Vector{Int8}}(undef, n_rec)
    accepted = 0
    step = 0
    rec  = 0

    while rec < n_rec
        step += 1
        idxs = randperm(N)[1:batch_size]
        old  = copy(spins)
        for i in idxs
            spins[i] = -spins[i]
        end
        dE = total_energy(spins, model) - E
        if dE <= 0 || rand() < exp(-dE/T)
            E += dE
            accepted += 1
        else
            spins .= old
        end

        if step % thin == 0
            rec += 1
            samples[rec] = copy(spins)
        end
    end

    println("Acceptance (post-burn): ", accepted/(total_steps - burn_in))
    return samples
end

#####################
# 4. Block-Spin + RG
#####################

function block_spin_transform_1d(spins::Vector{Int8})
    N = length(spins)
    @assert iseven(N) "Lattice size must be even"
    spins_new = Vector{Int8}(undef, N ÷ 2)

    for i in 1:(N ÷ 2)
        ssum = spins[2i-1] + spins[2i]
        if ssum > 0
            spins_new[i] = 1
        elseif ssum < 0
            spins_new[i] = -1
        else
            # tie: choose ±1 at random
            spins_new[i] = rand(Bool) ? Int8(1) : Int8(-1)
        end
    end

    return spins_new
end

function run_rg_procedure_ising_1d(
    initial_model::IsingModel1D,
    samples::Vector{Vector{Int8}},
    n_rg_steps::Int
)
    current_model   = initial_model
    Js, hs          = [current_model.J], [current_model.h]
    current_samples = samples

    for step in 1:n_rg_steps
        println("\n--- RG Step $step ---")
        println(" Number of samples: ", length(current_samples))

        # 1) block-spin transform each config
        transformed = [block_spin_transform_1d(s) for s in current_samples]

        # 2) re-estimate (pseudo-likelihood) on the new configs
        J_est, h_est = estimate_pl(transformed)

        println(" Estimated: J = $J_est, h = $h_est")

        # 3) update model & record
        current_model = IsingModel1D(current_model.lattice_size ÷ 2,
                                     J_est, h_est)
        push!(Js, J_est)
        push!(hs, h_est)

        # 4) move to next generation of samples
        current_samples = transformed
    end

    return Js, hs
end

#####################
# 5. Example Main
#####################

Random.seed!(1234)
N = 32
true_J, true_h = 0.35, 0.15
model = IsingModel1D(N, true_J, true_h)

println("Sampling…")
data = mcmc_samples(model, 1.0;
                    total_steps=50_000,
                    burn_in=10_000,
                    thin=1,
                    batch_size=4)

Js, hs = run_rg_procedure_ising_1d(model, data, 5)
